Importing Pandas for visualization, for what we are doing using PySpark to create an RDD on a HDFS will suit our purposes better due to file size and processing times.

In [34]:
import pandas as pd
from IPython.display import display

I hand keyed the first time stamp for simplicity. Pandas does not handle JSON files very well.

In [35]:
data_dict = {
    'time_id': ('09:32:01'),
    'current_total': (1385),
    'current_profit': (-30),
    'total_buy_price': (1415),
    'call_id': (1),
    'put_id': (1)
}

call_dict = {
                "time_id": ('09:32:01'),
                "call_id": (1),
                "current_total": (0),
                "current_profit": (0),
                "qty": (0),
                "buy_count": (0),
                "target_percent": (0),
                "total_buy_price": (0)
            }

put_dict = {
                "time_id": ('09:32:01'),
                "put_id": (1),
                "current_total": (1385),
                "current_profit": (-30),
                "qty": (2),
                "buy_count": (1),
                "target_percent": (65),
                "total_buy_price": (1415)
}

ticker_dict_calls = {
              "time_id": [("09:32:01"), ("09:32:01"), ("09:32:01"), ("09:32:01")],
              "ticker_symbol": [('AMD'), ('AMD~2'), ('NVDA'), ('NVDA~2')],
              "in_trade": [(False), (False), (False), (False)],
              "trade_closed": [(True), (True), (True), (True)],
              "dd_executed": [(False), (False), (False), (False)],
              "dd_position": [(0), (0), (0), (0)],
              "current_profit": [(0), (0), (0), (0)],
              "total_in": [(0), (0), (0), (0)],
              "max_percentage": [(0), (0), (0), (0)],
              "above_profit_target": [(False), (False), (False), (False)],
              "profit_target_reached": [(False), (False), (False), (False)],
              "alt_ticker": [("AMD~2"), ("AMD~2"), ("AMD~2"), ("AMD~2")],
              "last_5": [([]), ([]), ([]), ([])]
}

ticker_dict_puts = {
              "time_id": [("09:32:01"), ("09:32:01"), ("09:32:01"), ("09:32:01")],
              "ticker_symbol": [('AMD'), ('AMD~2'), ('NVDA'), ('NVDA~2')],
              "in_trade": [(False), (False), (True), (False)],
              "trade_closed": [(False), (True), (False), (True)],
              "dd_executed": [(False), (False), (False), (False)],
              "dd_position": [(0), (0), (0), (0)],
              "current_profit": [-(5), (0), -(25),(0)],
              "total_in": [(455), (0), (960), (0)],
              "max_percentage": [(0), (0), (0), (0)],
              "above_profit_target": [(False), (False), (False), (False)],
              "profit_target_reached": [(False), (False), (False), (False)],
              "alt_ticker": [("AMD~2"), ("AMD~2"), ("AMD~2"), ("AMD~2")],
              "last_5": [([]), ([]), ([]), ([])]
}

ticker_trend_dict = {
              "ticker_symbol": (('AMD'), ('AMD~2'), ('NVDA'), ('NVDA~2')),
              "time_id": (("09:32:01"), ("09:32:01"), ("09:32:01"), ("09:32:01")),
              "call_count": ((0), (0), (0), (0)),
              "put_count": ((0), (0), (0), (0))
}

Create DataFrames from the Dictionaries

In [36]:
data = pd.DataFrame.from_dict(data_dict, orient='index').transpose()
calls = pd.DataFrame.from_dict(call_dict, orient='index').transpose()
puts = pd.DataFrame.from_dict(put_dict, orient='index').transpose()
ticker_calls = pd.DataFrame.from_dict(ticker_dict_calls, orient='index').transpose()
ticker_puts = pd.DataFrame.from_dict(ticker_dict_puts, orient='index').transpose()
ticker_trend = pd.DataFrame.from_dict(ticker_trend_dict, orient='index').transpose()

Set options to allow us to see all the columns at once

In [37]:
pd.set_option('display.max_columns', None)

Printing our DataFrames 

In [38]:
display(data)
display(calls)
display(puts)
display(ticker_calls)
display(ticker_puts)

,time_id,current_total,current_profit,total_buy_price,call_id,put_id
0,09:32:01,1385,-30,1415,1,1


,time_id,call_id,current_total,current_profit,qty,buy_count,target_percent,total_buy_price
0,09:32:01,1,0,0,0,0,0,0


,time_id,put_id,current_total,current_profit,qty,buy_count,target_percent,total_buy_price
0,09:32:01,1,1385,-30,2,1,65,1415


,time_id,ticker_symbol,in_trade,trade_closed,dd_executed,dd_position,current_profit,total_in,max_percentage,above_profit_target,profit_target_reached,alt_ticker,last_5
0,09:32:01,AMD,False,True,False,0,0,0,0,False,False,AMD~2,[]
1,09:32:01,AMD~2,False,True,False,0,0,0,0,False,False,AMD~2,[]
2,09:32:01,NVDA,False,True,False,0,0,0,0,False,False,AMD~2,[]
3,09:32:01,NVDA~2,False,True,False,0,0,0,0,False,False,AMD~2,[]


,time_id,ticker_symbol,in_trade,trade_closed,dd_executed,dd_position,current_profit,total_in,max_percentage,above_profit_target,profit_target_reached,alt_ticker,last_5
0,09:32:01,AMD,False,False,False,0,-5,455,0,False,False,AMD~2,[]
1,09:32:01,AMD~2,False,True,False,0,0,0,0,False,False,AMD~2,[]
2,09:32:01,NVDA,True,False,False,0,-25,960,0,False,False,AMD~2,[]
3,09:32:01,NVDA~2,False,True,False,0,0,0,0,False,False,AMD~2,[]


This is a basic layout of what we want our DataFrames to look like. Using PySpark for the storage and management will allow the DataFrames to be accessed faster. Now there will be some aspects of the data intake we will need to address specifically the formatting of how the JSON spits out the data. PySpark does a pretty good job figuring out a schema based on what it is fed, however if there are redundancies it can get really confusing and makes the wrangling and cleaning a lot harder.